In [1]:
import numpy as np
import tensorflow as tf

from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Convolution2D, AveragePooling2D
from keras.layers import Input, Activation, Lambda
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.applications.resnet50 import identity_block, conv_block
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras import optimizers


Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5110)
/home/mzhao/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
def load_processed_fish(pathname, target_size):
    num_images = 3000
    label_bounds = [0, 199, 1910, 2641, 2758, 2933, 3000] #not using other fish
    label_counts = [label_bounds[i+1]-label_bounds[i] for i in range(len(label_bounds)-1)]
    
    fish_images = []
    for label in range(len(label_counts)):
        images = np.empty((label_counts[label], 224, 224, 3))
        for i in range(label_counts[label]):
            images[i,:,:,:] = skimage.io.imread(os.path.join(pathname, 
                "img_{0}label_{1}.jpg".format(i + label_bounds[label], label)))
        resize_image_set(images, target_size)
        fish_images.append(images)

    X = np.empty((6*target_size, 224, 224, 3))
    index = 0
    for fish in fish_images:
        for i in range(len(fish)):
            X[index] = fish[i]
            index += 1


    #.reshape((-1,224,224,3))
    y = []
    for i in range(len(label_counts)):
        y += [i] * target_size
    y = np.array(y)

    print "Loaded training images with shape {0}.".format(X.shape)
    print "Loaded training labels with shape {0}.".format(y.shape)
    np.save('samples.npy', X)
    np.save('labels.npy', y)

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    shuffled_images = X[indices]
    shuffled_labels = y[indices]
    split = int(X.shape[0]*.9)
    training_images = shuffled_images[:split]
    training_labels = shuffled_labels[:split]

    val_images = shuffled_images[split:]
    val_labels = shuffled_labels[split:]

    return training_images, training_labels, val_images, val_labels

In [2]:
px_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def preprocess(x):
    x = x - px_mean
    return x[:, ::-1]
#     return tf.reverse(x, [False, True, False, False])

In [3]:
size = (270, 480)
n_classes = 8
lr = 0.001
batch_size = 64
dropout = 0.4
weights_file = 'vgg16.h5'

model = Sequential()
model.add(Lambda(preprocess, input_shape=(3,) + size))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1000, activation='softmax'))

model.load_weights(weights_file)
model.pop(); model.pop(); model.pop()

for layer in model.layers:
    layer.trainable = False

model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adadelta", metrics=["accuracy"])

In [4]:
def inception_preprocess(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

size = (299, 299)
n_classes = 8
lr = 0.001
batch_size = 64

img_input = Input(shape=(3, size[0], size[1]))
inception = InceptionV3(include_top=False, weights='imagenet', input_tensor=img_input)

for layer in inception.layers:
    layer.trainable = False

output = inception.output
output = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(output)
output = Flatten(name='flatten')(output)
output = Dense(n_classes, activation='softmax', name='predictions')(output)

model = Model(inception.input, output)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9, decay=0.0, nesterov=True),
              metrics=["accuracy"])

In [4]:
trn_path = 'train/'
train_datagen = ImageDataGenerator()
trn_gen = train_datagen.flow_from_directory(trn_path, target_size=size, batch_size=50,
                                                    class_mode='categorical', shuffle=True)

# model.fit(X_train, Y_train, nb_epoch=5, batch_size=32)
# loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)

Found 3777 images belonging to 8 classes.


In [6]:
# inception net
nb_trn_samples = 3777
nb_epoch = 20

model.fit_generator(trn_gen, samples_per_epoch=nb_trn_samples, nb_epoch=nb_epoch, verbose=2)

Epoch 1/10
80s - loss: 2.1645 - acc: 0.1660
Epoch 2/10
79s - loss: 1.9904 - acc: 0.2568
Epoch 3/10
79s - loss: 1.8615 - acc: 0.3272
Epoch 4/10
79s - loss: 1.7833 - acc: 0.3691
Epoch 5/10
79s - loss: 1.7187 - acc: 0.4069
Epoch 6/10
79s - loss: 1.6561 - acc: 0.4440
Epoch 7/10
79s - loss: 1.6144 - acc: 0.4676
Epoch 8/10
79s - loss: 1.5815 - acc: 0.4739
Epoch 9/10
79s - loss: 1.5394 - acc: 0.4951
Epoch 10/10
79s - loss: 1.5039 - acc: 0.5068


In [5]:
# vgg
nb_trn_samples = 3777
nb_epoch = 20

model.fit_generator(trn_gen, samples_per_epoch=nb_trn_samples, nb_epoch=nb_epoch, verbose=2)

Epoch 1/20


MemoryError: Error allocating 1678131200 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuAlloc{memset_0=True}(CudaNdarrayConstant{0.0}, Shape_i{0}.0, TensorConstant{64}, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0)
Toposort index: 172
Inputs types: [CudaNdarrayType(float32, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), (), ()]
Inputs strides: [(), (), (), (), ()]
Inputs values: [CudaNdarray(0.0), array(50), array(64), array(272), array(482)]
Outputs clients: [[GpuIncSubtensor{InplaceSet;::, ::, int64:int64:, int64:int64:}(GpuAlloc{memset_0=True}.0, GpuElemwise{Composite{(i0 * ((i1 + i2) + Abs((i1 + i2))))},no_inplace}.0, Constant{1}, ScalarFromTensor.0, Constant{1}, ScalarFromTensor.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting th

In [19]:
# paths
train_path = '/train/'
# test_path = '/test/'
saved_model_path = '/models/'

# data
batch_size = 16
nb_split_train_samples = 3277
nb_full_train_samples = 3785
nb_valid_samples = 500
# nb_test_samples = 1000
classes = ["ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

# model
nb_runs = 5
nb_epoch = 30
nb_aug = 5
dropout = 0.4
clip = 0.01
use_val = False
archs = ["vggbn"]

models = {
    "vggbn": Vgg16BN(size=(270, 480), n_classes=nb_classes, lr=0.001,
                           batch_size=batch_size, dropout=dropout),
#     "inception": Inception(size=(299, 299), n_classes=nb_classes,
#                            lr=0.001, batch_size=batch_size),
#     "resnet": Resnet50(size=(270, 480), n_classes=nb_classes, lr=0.001,
#                     batch_size=batch_size, dropout=dropout)
}